In [6]:
import openai
import pyttsx3
import pyaudio
import wave
import random
from pinecone import Pinecone
from openai import OpenAI

openai.api_key = "sk-svcacct-40OsNDk5Z7NQe25GAoyV8HCAVizvFOSkAeC2fHNNnPqj3UdmyZeMVfRcMnhg4ZT3BlbkFJ-k-oq6yKeb5rcb0hTr8Ybiy9HG3Q_PSecyFFtg9mmXXedMk31urv5ykV7DUH8A"
pinecone_api_key = "pcsk_gToqV_QsdweU7dicHERcFNnnYhgo6hU1Dntnue5JWyuRV3ij8gsmqw8tvRS1syDshVPsj"

pc = Pinecone(api_key=pinecone_api_key)
index_name = "virtual-clinic"
index = pc.Index(index_name)

engine = pyttsx3.init()


In [7]:

def record_audio(file_path, duration=5):
    """Record audio and save it to the given file path."""
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    rate = 44100
    p = pyaudio.PyAudio()

    print("Recording...")
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk)
    frames = []

    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(sample_format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

    print("Recording finished.")


In [8]:

# Function to transcribe audio
def transcribe_audio(file_path):
    """Transcribe audio to text using OpenAI Whisper."""
    client = OpenAI(api_key ="sk-svcacct-40OsNDk5Z7NQe25GAoyV8HCAVizvFOSkAeC2fHNNnPqj3UdmyZeMVfRcMnhg4ZT3BlbkFJ-k-oq6yKeb5rcb0hTr8Ybiy9HG3Q_PSecyFFtg9mmXXedMk31urv5ykV7DUH8A")
    with open(file_path, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
          model="whisper-1", 
          file=audio_file
        )
    return transcription.text


In [9]:

# Function to speak text
def speak_text(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()



In [10]:
def query_pinecone(query_text):
    """Query Pinecone for patient information."""
    client = OpenAI(api_key="sk-svcacct-40OsNDk5Z7NQe25GAoyV8HCAVizvFOSkAeC2fHNNnPqj3UdmyZeMVfRcMnhg4ZT3BlbkFJ-k-oq6yKeb5rcb0hTr8Ybiy9HG3Q_PSecyFFtg9mmXXedMk31urv5ykV7DUH8A")

    
    response = client.embeddings.create(
        input=query_text,
        model="text-embedding-3-small" 
    )
    
    query_embedding = response.data[0].embedding 
    try:
        result = index.query(
            vector=query_embedding,
            top_k=1,
            include_metadata=True
        )
        if result["matches"]:
            match = result["matches"][0]
            metadata = match.get("metadata", {})
            return {
                "id": match["id"],
                "score": match["score"],
                "allergy": metadata.get("allergy", "None"),
                "medicine_to_avoid": metadata.get("medicine_to_avoid", "None"),
                "food_to_avoid": metadata.get("food_to_avoid", "None")
            }
    except Exception as e:
        print(f"Error querying Pinecone: {e}")
    return None


# Patient profiles
patient_profiles = {
    "child": "You are a 10-year-old child speaking to a doctor.",
    "elderly": "You are a 70-year-old patient who might not understand medical jargon.",
    "default": "You are a 30-year-old patient responding to a doctor's questions.",
    "teenager": "You are a 16-year-old who might feel hesitant to share details openly.",
    "busy_adult": "You are a busy professional who wants a quick resolution to health issues."
}
selected_profile = random.choice(list(patient_profiles.values()))
print(f"Selected Profile: {selected_profile}")



Selected Profile: You are a 10-year-old child speaking to a doctor.


In [11]:

def virtual_clinic():
    """Run the Virtual Clinic Simulation."""
    print("Starting Virtual Clinic Simulation. Speak your input into the microphone. Say 'quit' to exit.")
    while True:
        doctor_audio_path = "C:/Users/User/Python_/doctor_input.wav"
        record_audio(doctor_audio_path, duration=5)

        transcribed_input = transcribe_audio(doctor_audio_path)
        print("Doctor (Transcribed):", transcribed_input)

        if "quit" in transcribed_input.lower():
            print("Exiting simulation.")
            break

        patient_info = query_pinecone(transcribed_input)
        if patient_info:
            print(f"Matched Patient Data: {patient_info}")
            pinecone_data = (f"Allergy: {patient_info['allergy']}, "
                             f"Medicine to avoid: {patient_info['medicine_to_avoid']}, "
                             f"Food to avoid: {patient_info['food_to_avoid']}")
        else:
            pinecone_data = "No relevant patient data found."

        user_message = f"{selected_profile}\nDoctor: {transcribed_input}\nPatient Info: {pinecone_data}"
        client = OpenAI(api_key ="sk-svcacct-40OsNDk5Z7NQe25GAoyV8HCAVizvFOSkAeC2fHNNnPqj3UdmyZeMVfRcMnhg4ZT3BlbkFJ-k-oq6yKeb5rcb0hTr8Ybiy9HG3Q_PSecyFFtg9mmXXedMk31urv5ykV7DUH8A")
        completion = client.chat.completions.create(
            model="o1-mini-2024-09-12",
            messages=[
                {"role": "user", "content": user_message}
            ]
        )
        patient_response = completion.choices[0].message.content
        print("Patient:", patient_response)

        speak_text(patient_response)

virtual_clinic()


Starting Virtual Clinic Simulation. Speak your input into the microphone. Say 'quit' to exit.
Recording...
Recording finished.
Doctor (Transcribed): Hi, how are you today?
Matched Patient Data: {'id': 'elderly1', 'score': 0.212654352, 'allergy': 'Shellfish', 'medicine_to_avoid': 'Statins', 'food_to_avoid': 'Shellfish dishes'}
Patient: Hi, I'm doing okay, thank you. I have an allergy to shellfish, so I make sure not to eat any shellfish dishes. Also, I need to avoid medicines like statins. I just wanted to let you know so we can stay safe!
Recording...
Recording finished.
Doctor (Transcribed): Quit
Exiting simulation.


In [14]:
import openai
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key="pcsk_gToqV_QsdweU7dicHERcFNnnYhgo6hU1Dntnue5JWyuRV3ij8gsmqw8tvRS1syDshVPsj"  # Replace with your Pinecone API key
)

index_name = "virtual-clinic"

try:
    if index_name not in pc.list_indexes():
        pc.create_index(
            name=index_name,
            dimension=1536, 
            metric="cosine",  
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")
except Exception as e:
    print(f"Index creation error: {e}")

index = pc.Index(index_name)

patient_records = [
    {"id": "child1", "text": "I feel tired and have a sore throat. I don't want to go to school.",
     "allergy": "Peanuts", "medicine_to_avoid": "Ibuprofen", "food_to_avoid": "Peanut butter"},
    {"id": "child2", "text": "I hurt my arm while playing soccer yesterday.",
     "allergy": "None", "medicine_to_avoid": "None", "food_to_avoid": "None"},
    {"id": "elderly1", "text": "I’ve been feeling dizzy and weak lately. I have trouble climbing stairs.",
     "allergy": "Shellfish", "medicine_to_avoid": "Statins", "food_to_avoid": "Shellfish dishes"},
    {"id": "elderly2", "text": "I think my blood pressure medication might be making me feel light-headed.",
     "allergy": "Latex", "medicine_to_avoid": "ACE inhibitors", "food_to_avoid": "High-sodium foods"},
    {"id": "default1", "text": "I’ve had a persistent headache for two days and some mild nausea.",
     "allergy": "Pollen", "medicine_to_avoid": "Aspirin", "food_to_avoid": "Raw honey"},
    {"id": "default2", "text": "I feel shortness of breath and chest tightness during physical activities.",
     "allergy": "Dust mites", "medicine_to_avoid": "Beta blockers", "food_to_avoid": "Dairy products"},
    {"id": "teenager1", "text": "I have acne and feel self-conscious about it. What can I do to treat it?",
     "allergy": "None", "medicine_to_avoid": "Steroids", "food_to_avoid": "Fried foods"},
    {"id": "teenager2", "text": "I’ve been feeling anxious about exams and can’t sleep well.",
     "allergy": "None", "medicine_to_avoid": "None", "food_to_avoid": "Caffeinated drinks"},
    {"id": "busy_adult1", "text": "I need a quick remedy for this cold. I can’t afford to take time off work.",
     "allergy": "Aspirin", "medicine_to_avoid": "NSAIDs", "food_to_avoid": "Alcohol"},
    {"id": "busy_adult2", "text": "I have neck pain from working long hours at a desk.",
     "allergy": "None", "medicine_to_avoid": "None", "food_to_avoid": "None"}
]

openai.api_key = "sk-svcacct-40OsNDk5Z7NQe25GAoyV8HCAVizvFOSkAeC2fHNNnPqj3UdmyZeMVfRcMnhg4ZT3BlbkFJ-k-oq6yKeb5rcb0hTr8Ybiy9HG3Q_PSecyFFtg9mmXXedMk31urv5ykV7DUH8A"

try:
    for record in patient_records:
        combined_text = (f"{record['text']} Allergy: {record.get('allergy', 'None')}, "
                         f"Medicine to avoid: {record.get('medicine_to_avoid', 'None')}, "
                         f"Food to avoid: {record.get('food_to_avoid', 'None')}")
        embedding = openai.Embedding.create(
            input=combined_text,
            model="text-embedding-3-small"
        )["data"][0]["embedding"]

        
        index.upsert([{"id": record["id"], "values": embedding, "metadata": record}])
        print(f"Record {record['id']} inserted into Pinecone.")
except Exception as e:
    print(f"Error generating embeddings or inserting into Pinecone: {e}")
query_text = "What should a patient allergic to peanuts avoid?"
client = OpenAI(api_key="sk-svcacct-40OsNDk5Z7NQe25GAoyV8HCAVizvFOSkAeC2fHNNnPqj3UdmyZeMVfRcMnhg4ZT3BlbkFJ-k-oq6yKeb5rcb0hTr8Ybiy9HG3Q_PSecyFFtg9mmXXedMk31urv5ykV7DUH8A")
# Generate embeddings
query_embedding = client.embeddings.create(
    input=query_text,
    model="text-embedding-3-small" 
).data[0].embedding  

try:
    result = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )
    for match in result["matches"]:
        if "metadata" in match:
            metadata = match["metadata"]
            print(f"Match ID: {match['id']}, Score: {match['score']}")
            print(f"Allergy: {metadata['allergy']}, Medicine to avoid: {metadata['medicine_to_avoid']}, Food to avoid: {metadata['food_to_avoid']}")
        else:
            print(f"Match ID: {match['id']}, Score: {match['score']}, Metadata not found.")
except Exception as e:
    print(f"Error querying Pinecone: {e}")


Index creation error: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'X-Cloud-Trace-Context': '8410c37f0f9a632e56879b8709bb75ec', 'Date': 'Thu, 28 Nov 2024 22:13:08 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}

Error generating embeddings or inserting into Pinecone: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you 

In [20]:
!pip install langchain_community
!pip install langchain

!pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/04/fc/6f52588ac1cb4400a7804ef88d0d4e00cfe57a7ac6793ec3b00de5a8758b/pypdf-5.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   ------------ --------------------------- 92.2/298.0 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------  297.0/298.0 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 298.0/298.0 kB 3.1 MB/s eta 0:00:00


In [22]:
%pip install --upgrade --quiet faiss-cpu 

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
fitter 1.7.0 requires numpy<2.0.0,>=1.20.0, but you have numpy 2.1.3 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.1.3 which is incompatible.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.1.3 which is incompatible.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.1.3 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.1.3 which is incompatible.


In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "C:\\Users\\User\\Python_\\problems-solutions .pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

pages[0]

Document(page_content='[ \n    { \n        "id": "child1", \n        "text": "I have a skin rash. What could it be and how should I treat it?", \n        "allergy": "None", \n        "medicine_to_avoid": "None", \n        "food_to_avoid": "Sugary and processed foods" \n    }, \n    { \n        "id": " child2", \n        "text": "I have small bumps on my body  and face What could be causing this?", \n        "allergy": "None", \n        "medicine_to_avoid": " Steroids", \n        "food_to_avoid": " None " \n    }, \n    { \n        "id": "child3", \n        "text": "I feel tired and have a sore throat”, \n        "allergy": "Peanuts", \n        "medicine_to_avoid": "Ibuprofen", \n        "food_to_avoid": "Peanut products" \n    }, \n \n    { \n        "id": "default1", \n        "text": "I experience allergies frequently. Could you explain common triggers and treatments?", \n        "allergy": "Shellfish", \n        "medicine_to_avoid": "Statins", \n        "food_to_avoid": "Shellfish d

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:········


In [25]:
!pip install langchain_openai

  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/c3/36/a5426bf8195af460aefefe9c15bbf091a549cfd9cb243efd167d6a7b3a42/langchain_openai-0.2.10-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.4.0,>=0.3.21 from https://files.pythonhosted.org/packages/9b/50/e0bd90fc481d1cc8c2039ad6161b20fc8e396a7cba6064a4f1e8e5afea62/langchain_core-0.3.21-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken<1,>=0.7 from https://files.pythonhosted.org/packages/1e/86/eea2309dc258fb86c7d9b10db536434fc16420feaa3b6113df18b23db7c2/tiktoken-0.8.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tiktoken-0.8.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Obtaining dependency information for langsmith<0.2.0,>=0.1.125 from https://files.pythonhosted.org/packages/de/f0/63b06b99b730b9954f8709f6f7d9b8d076fa0a973e472efe278089bde42b/langsmith-0.1.147-py3-none-any.whl.metadata
  Obtaining dependency information for p

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\~iktoken\\_tiktoken.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("What are the patients allergies?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

ModuleNotFoundError: No module named 'langchain_openai'